In [2]:
pip install asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 2.9 MB/s eta 0:00:00


In [ ]:
pip install aiohttp

In [ ]:
pip install nest_asyncio


In [ ]:
pip install pandas

In [39]:
import aiohttp
import asyncio
import os
import pandas as pd
from PIL import Image
from io import BytesIO

In [ ]:
async def get_image_size(session, url):
    try:
      async with session.get(url) as response:
        if response.status == 200:
            image_data = await response.read()
            image = Image.open(BytesIO(image_data))
            return image.size
    except Exception as e:
      pass

In [ ]:
async def process_images(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [get_image_size(session, url) for url in urls]
        image_sizes = await asyncio.gather(*tasks)
        return image_sizes

# Загрузка данных из таблицы
df = pd.read_csv('/content/Parser_ImageSize - feed.csv')
urls = df['image_url'].tolist()

In [ ]:
task_pool = 1000

bunches = [urls[i:i+task_pool] for i in range(0, len(urls), task_pool)]

image_sizes = []
for bunch in bunches:
    results = asyncio.run(process_images(bunch))
    image_sizes.extend(results)

In [ ]:
widths = []
heights = []

for size in image_sizes:
    if size:
        widths.append(size[0])
        heights.append(size[1])
    else:
        widths.append(None)
        heights.append(None)

In [ ]:
widths_x_heights = [f"{w}x{h}" if w is not None and h is not None else None for w, h in zip(widths, heights)]

df['SIZE'] = widths_x_heights

df.to_csv('/content/Parser_ImageSize - feed.csv', index=False)